In [9]:
# %pip install pandas
# %pip install xlrd
import pandas as pd
import re

In [10]:
df = pd.read_excel("cmed_v4.xlsx")
df.head(1)

,Unnamed: 0,0,1,2,3,4,5,6,7,8,9,10,11,12,rec,13,Por subs (1 a 4),Por tipo(1 a 10),Probabilidade,Validade
0,22,359,AMPICILINA,57.507.378/0003-65,EMS S/A,507701907133117,1023504310059,AMPICILINA,50 MG/ML PO P/ SUS OR CT FR VD AMB X 150 ML,150,J1C1 - PENICILINAS ORAIS DE AMPLO ESPECTRO,Genérico,53.61,74.11,AM,Tarja Vermelha (**),1,1,2.5,180


In [11]:
# selecionar somente linhas desejadas
df = df.filter(items=[1, 2, 3, 4, 6, 7, 9, 10, 11, 12, 'rec', 'Probabilidade'])

# renomear algumas colunas
df = df.rename(columns={1: 'SUBSTÂNCIA', 2: 'CNPJ', 3: 'LABORATÓRIO', 4: 'GGREM', 6: 'PRODUTO', 7: 'APRESENTAÇÃO', 9: 'CLASSE TERAPÊUTICA', 10: 'TIPO', 11: 'CUSTO', 12: 'PREÇO', 'rec': 'CONTROLE', 'Probabilidade': 'PROBABILIDADE'})
df.head()

,SUBSTÂNCIA,CNPJ,LABORATÓRIO,GGREM,PRODUTO,APRESENTAÇÃO,CLASSE TERAPÊUTICA,TIPO,CUSTO,PREÇO,CONTROLE,PROBABILIDADE
0,AMPICILINA,57.507.378/0003-65,EMS S/A,507701907133117,AMPICILINA,50 MG/ML PO P/ SUS OR CT FR VD AMB X 150 ML,J1C1 - PENICILINAS ORAIS DE AMPLO ESPECTRO,Genérico,53.61,74.11,AM,2.5
1,BENZOATO DE ALOGLIPTINA;CLORIDRATO DE METFORMINA,60.397.775/0001-74,TAKEDA PHARMA LTDA.,501115110025705,NESINA MET,"(12,5 + 1000,0) MG COM REV CT BL AL AL X 60",A10N3 - ASSOCIAÇÕES DE INIBIDORES DPP-IV COM B...,Novo,228.00,303.77,Liberado,2.5
2,CARBAMAZEPINA,56.994.502/0001-30,NOVARTIS BIOCIENCIAS S.A,526515003110311,TEGRETOL,200 MG COM CT BL AL PLAS PVC TRANS X 60,N3A - ANTIEPILÉPTICOS,Novo,64.54,89.22,C1,2.5
3,CLORIDRATO DE SERTRALINA,73.856.593/0001-66,PRATI DONADUZZI & CIA LTDA,528519120163307,TRELIM,100 MG COM REV CT BL AL PLAS PVC/PVDC TRANS X 60,N6A4 - ANTIDEPRESSIVOS SSRI,Similar,290.24,401.24,C1,2.5
4,EZETIMIBA,48.344.725/0007-19,ALTHAIA S.A INDÚSTRIA FARMACÊUTICA,543817070024603,ZETERINA,10 MG COM CT BL AL PLAS PVDC/PVC TRANS X 60,C10A9 - TODOS OUTROS REGULADORES DE COLESTEROL...,Similar,200.62,277.35,Liberado,2.5


## Criando colunas "DOSAGEM", "APRESENTACAO", "QUANTIDADE", "CLASSE"

In [12]:
ps = ['MG\s\/\sML', 'MEQ/L', 'ML/ML', 'MG/ML', 'MG/G', 'GV/ML', 'G/L', 'G/G', 'MG\)', 'MG', 'MCG', 'PCC', 'G/ML', 'TRU', 
      'MU/ML', 'UI/ML', 'SOL\sINJ', 'UI/G', 'U/G', 'U/U', '\sUI\s', '\sML\s', '\sU\s', '\sG\s', '\s1G\s', '1G\s', '$']
pt3 = r'POM|CREM|GEL|MOLE|EMU|PAST|LOCAO|XAMP'
pt4 = r'SOL|XPE|SUS|INJ|ML|IM'
pt5 = r' - '

dosagem = []
apresentacao = []
classe = []
quantidade = []

for i in range(df.shape[0]):
  a = df.loc[i, 'APRESENTAÇÃO'].replace(' ', '')
  
  # DOSAGEM
  d = ''
  for p in ps:
    m1 = re.search(p, a)
    if m1:
      d = re.split(p, a)[0] + m1.group()
      break
  d = d.strip()
  
  # QUANTIDADE
  m2 = re.split('X\s', a)
  q = m2[-1] if m2 else ''
  q = q.strip()

  # APRESENTACAO
  m3 = re.search(pt3, a)
  m4 = re.search(pt4, a)
  if m3: t = 'Semissólido'
  elif m4: t = 'Líquido'
  else: t = 'Sólido'

  dosagem.append(d)
  quantidade.append(q)
  apresentacao.append(t)
  # print(a, '=> DOSAGEM:', d, '| QUANTIDADE:', q, '| APRESENTACAO:', t)

  a = df.loc[i, 'CLASSE TERAPÊUTICA'].replace(' ', '')
  cl = re.split(pt5, a)[1]
  classe.append(cl)
  # print(a, '=> CONTROLE:', co, '| CLASSE:', cl)
  
df['DOSAGEM'] = dosagem
df['APRESENTACAO'] = apresentacao
df['QUANTIDADE'] = quantidade
df['CLASSE'] = classe

In [13]:
df.head()

,SUBSTÂNCIA,CNPJ,LABORATÓRIO,GGREM,PRODUTO,APRESENTAÇÃO,CLASSE TERAPÊUTICA,TIPO,CUSTO,PREÇO,CONTROLE,PROBABILIDADE,DOSAGEM,APRESENTACAO,QUANTIDADE,CLASSE
0,AMPICILINA,57.507.378/0003-65,EMS S/A,507701907133117,AMPICILINA,50 MG/ML PO P/ SUS OR CT FR VD AMB X 150 ML,J1C1 - PENICILINAS ORAIS DE AMPLO ESPECTRO,Genérico,53.61,74.11,AM,2.5,50 MG/ML,Líquido,150 ML,PENICILINAS ORAIS DE AMPLO ESPECTRO
1,BENZOATO DE ALOGLIPTINA;CLORIDRATO DE METFORMINA,60.397.775/0001-74,TAKEDA PHARMA LTDA.,501115110025705,NESINA MET,"(12,5 + 1000,0) MG COM REV CT BL AL AL X 60",A10N3 - ASSOCIAÇÕES DE INIBIDORES DPP-IV COM B...,Novo,228.00,303.77,Liberado,2.5,"(12,5 + 1000,0) MG",Sólido,60,ASSOCIAÇÕES DE INIBIDORES DPP-IV COM BIGUANIDAS
2,CARBAMAZEPINA,56.994.502/0001-30,NOVARTIS BIOCIENCIAS S.A,526515003110311,TEGRETOL,200 MG COM CT BL AL PLAS PVC TRANS X 60,N3A - ANTIEPILÉPTICOS,Novo,64.54,89.22,C1,2.5,200 MG,Sólido,60,ANTIEPILÉPTICOS
3,CLORIDRATO DE SERTRALINA,73.856.593/0001-66,PRATI DONADUZZI & CIA LTDA,528519120163307,TRELIM,100 MG COM REV CT BL AL PLAS PVC/PVDC TRANS X 60,N6A4 - ANTIDEPRESSIVOS SSRI,Similar,290.24,401.24,C1,2.5,100 MG,Sólido,60,ANTIDEPRESSIVOS SSRI
4,EZETIMIBA,48.344.725/0007-19,ALTHAIA S.A INDÚSTRIA FARMACÊUTICA,543817070024603,ZETERINA,10 MG COM CT BL AL PLAS PVDC/PVC TRANS X 60,C10A9 - TODOS OUTROS REGULADORES DE COLESTEROL...,Similar,200.62,277.35,Liberado,2.5,10 MG,Sólido,60,TODOS OUTROS REGULADORES DE COLESTEROL E TRIGL...


# Inserts

In [14]:
import psycopg2

# cria conexão
def conn():
  con = psycopg2.connect(
    database = "drogasystem", 
    user = "oscar", 
    password = "oscar", 
    host = "127.0.0.1", 
    port = "5432")
  cur = con.cursor()
  return con, cur

# faz selects
def select(sql):
  all = []
  con, cur = conn()
  cur.execute(sql)
  all = cur.fetchall()
  cur.close()
  con.close()
  return all

# executa inserts e updates
def exec(sql):
  try:
    con, cur = conn()
    cur.execute(sql)
    con.commit()
  except (Exception, psycopg2.Error) as error:
    if cur == None: cur.rollback()
    print("Error:", error)
  finally:
    if cur: cur.close()
    if con: con.close()


In [15]:
# nome laboratório
df_lab = df.filter(items=['CNPJ', 'LABORATÓRIO']).drop_duplicates(subset=['CNPJ']).reset_index(drop=True)

nome = []
for e in df_lab['LABORATÓRIO']:
  s = e
  s = s.replace('LABORATÓRIOS', '').strip()
  s = s.replace('LABORATÓRIO', '').strip()
  s = s.replace('LABORATORIO', '').strip()
  s = s.replace('FARMACÊUTICO', '').strip()
  s = s.replace('FARMACÊUTICA', '').strip()
  s = s.replace('INDUSTRIAL', '').strip()
  s = s.replace('INDÚSTRIA', '').strip()
  s = s.replace('INDUSTRIA', '').strip()
  s = s.replace('PRODUTOS', '').strip()
  s = s.replace('PRODUTOS', '').strip()
  s = s[:3] + re.split(r'\.|\s', s[3:])[0]
  s = s.replace('  ', ' ').strip()
  # print(s, '=>', e)
  nome.append(s)
df_lab['NOME'] = nome
df_lab.head(10)

# insert laboratório
sql = f"insert into laboratorio values \n  (default, '{df_lab.iloc[0, 2]}', '{df_lab.iloc[0, 1]}', '{df_lab.iloc[0, 0]}')"
for i in range(1, df_lab.shape[0]):
  sql += f", \n  (default, '{df_lab.iloc[i, 2]}', '{df_lab.iloc[i, 1]}', '{df_lab.iloc[i, 0]}')"
sql += ";"

print(sql)
exec(sql)

insert into laboratorio values 
  (default, 'EMS', 'EMS S/A', '57.507.378/0003-65'), 
  (default, 'TAKEDA', 'TAKEDA PHARMA LTDA.', '60.397.775/0001-74'), 
  (default, 'NOVARTIS', 'NOVARTIS BIOCIENCIAS S.A', '56.994.502/0001-30'), 
  (default, 'PRATI', 'PRATI DONADUZZI & CIA LTDA', '73.856.593/0001-66'), 
  (default, 'ALTHAIA', 'ALTHAIA S.A INDÚSTRIA FARMACÊUTICA', '48.344.725/0007-19'), 
  (default, 'PFIZER', 'LABORATÓRIOS PFIZER LTDA', '46.070.868/0036-99'), 
  (default, 'ACHÉ', 'ACHÉ LABORATÓRIOS FARMACÊUTICOS S.A', '60.659.463/0029-92'), 
  (default, 'ASTRAZENECA', 'ASTRAZENECA DO BRASIL LTDA', '60.318.797/0001-00'), 
  (default, 'BOEHRINGER', 'BOEHRINGER INGELHEIM DO BRASIL QUÍMICA E FARMACÊUTICA LTDA.', '60.831.658/0001-77'), 
  (default, 'EUROFARMA', 'EUROFARMA LABORATÓRIOS S.A.', '61.190.096/0001-92'), 
  (default, 'GLAXOSMITHKLINE', 'GLAXOSMITHKLINE BRASIL LTDA', '33.247.743/0001-10'), 
  (default, 'SANDOZ', 'SANDOZ DO BRASIL INDÚSTRIA FARMACÊUTICA LTDA', '61.286.647/0001-16'),

In [16]:
# seleciona laboratório
sql = "select * from laboratorio order by 1;"
df_lab = pd.DataFrame(select(sql), columns=['id', 'nome', 'razao', 'cnpj'])
# df_lab.head()

# insert medicamento
laboratorio_id = df_lab[df_lab['cnpj'] == df.iloc[0, 1]]['id'].reset_index(drop=True)[0]
sql  = f"insert into medicamento values \n"
sql += f"  (default, '{laboratorio_id}', '{df.iloc[0, 4]}', '{df.iloc[0, 0]}', '{df.iloc[0, 12]}', '{df.iloc[0, 13]}', '{df.iloc[0, 7]}'"
sql += f", '{df.iloc[0, 10]}', '{df.iloc[0, 14]}', '{df.iloc[0, 3]}', '{df.iloc[0, 15]}', '{df.iloc[0, 9]}')"
for i in range(1, df.shape[0]):
  laboratorio_id = df_lab[df_lab['cnpj'] == df.iloc[i, 1]]['id'].reset_index(drop=True)[0]
  sql += f", \n  (default, '{laboratorio_id}', '{df.iloc[i, 4]}', '{df.iloc[i, 0]}', '{df.iloc[i, 12]}', '{df.iloc[i, 13]}', '{df.iloc[i, 7]}'"
  sql += f", '{df.iloc[i, 10]}', '{df.iloc[i, 14]}', '{df.iloc[i, 3]}', '{df.iloc[i, 15]}', '{df.iloc[i, 9]}')"
sql += ";"

print(sql)
exec(sql)

insert into medicamento values 
  (default, '1', 'AMPICILINA', 'AMPICILINA', '50 MG/ML', 'Líquido', 'Genérico', 'AM', '150 ML', '507701907133117', 'PENICILINAS ORAIS DE AMPLO ESPECTRO', '74.11'), 
  (default, '2', 'NESINA MET', 'BENZOATO DE ALOGLIPTINA;CLORIDRATO DE METFORMINA', '(12,5 + 1000,0) MG', 'Sólido', 'Novo', 'Liberado', '60', '501115110025705', 'ASSOCIAÇÕES DE INIBIDORES DPP-IV COM BIGUANIDAS', '303.77'), 
  (default, '3', 'TEGRETOL', 'CARBAMAZEPINA', '200 MG', 'Sólido', 'Novo', 'C1', '60', '526515003110311', 'ANTIEPILÉPTICOS', '89.22'), 
  (default, '4', 'TRELIM', 'CLORIDRATO DE SERTRALINA', '100 MG', 'Sólido', 'Similar', 'C1', '60', '528519120163307', 'ANTIDEPRESSIVOS SSRI', '401.24'), 
  (default, '5', 'ZETERINA', 'EZETIMIBA', '10 MG', 'Sólido', 'Similar', 'Liberado', '60', '543817070024603', 'TODOS OUTROS REGULADORES DE COLESTEROL E TRIGLICERÍDEOS', '277.35'), 
  (default, '6', 'CITALOR', 'ATORVASTATINA CÁLCICA', '10 MG', 'Sólido', 'Novo', 'Liberado', '60', '5528200500682